# FDTD testing notebook

### Description

This notebook implements simple tests of the FDTD solver class written in Python.

Following tests are implemented:

1. Eigenmodes of a parallelepiped cavity with perfect cundoctor as a boundary.
2. Visual inspection of an initialization of the electric field to that of a point charge.
3. Visual inspection of an initialization of the electric field to that of a point charge *but* with the origin of the grid shifted by a given amount.

## Tests

Necessary imports of general libraries

In [ ]:
import numpy as np
import scipy as sp

%matplotlib widget

from matplotlib import pyplot as plt

import holoviews as hv
from scipy import constants

Import the grid class. This class does not diffeentiate between the field components in the array allocation and size.

In [ ]:
from fdtd3d import UnifiedYeeGrid

Define physical quantities in SI units. This can be circumvented using `scipy.constants`

In [ ]:
eps0 = constants.epsilon_0 #8.8541878e-12;
mu0 = constants.mu_0 #4e-7 * np.pi;
c0= constants.c #299792458;

We define a cavity of dimensions $L_x \times L_y \times L_z$ and discretize it using $N_x \times N_y \times N_z$ cells.

The fields will be initialized to white noise and propagated for $N_t$ time steps. Using the resulting sampled time-series of the electric field we can compute the empirical eigenfrequencies of the cavity, to be compared with TE and TM modes which can be determined analytically.

In [ ]:
Lx = .05; 
Ly = .04; 
Lz = .03;
Nx = 25; 
Ny = 20; 
Nz = 15; 
Cx = Nx / Lx; 
Cy = Ny / Ly; 
Cz = Nz / Lz; 
Nt = 4096;
Dt = 1/(c0*np.linalg.norm([Cx, Cy, Cz]))

In [ ]:
grid = UnifiedYeeGrid(Nx, Ny, Nz, pmlCells=(0,0,0), cellDim=(1./Cx, 1./Cy, 1./Cz) )

In [ ]:
grid.initRandom()

In [ ]:
Et = np.zeros((Nt,3));

In [ ]:
for t in range(Nt):
            grid.updateOwn(eps0, mu0, Dt, (1./Cx, 1./Cy, 1./Cz) );
            Et[t, :] = np.array([ grid.Ex(3,3,3), grid.Ey(3,3,3), grid.Ez(3,3,3) ])

In [ ]:
t = np.linspace(0, Nt*Dt, Nt)

In [ ]:
plt.plot(t, Et[:,0], 'r-',  t, Et[:,1], 'g-', t, Et[:,2], 'b')
plt.legend(("Ex", "Ey", "Ez"), loc='best')

In [ ]:
plt.close()

In [ ]:
plt.semilogy(t, np.linalg.norm(Et, axis=1)**2, 'k-')

In [ ]:
ftFields = np.fft.fft(Et, axis=0)

In [ ]:
ftFields.shape

In [ ]:
frequencies=np.fft.fftfreq(Et.shape[0], d=Dt)

In [ ]:
angularFrequencies=2*np.pi*frequencies

In [ ]:
plt.close()

In [ ]:
plt.plot(np.fft.fftshift(angularFrequencies), np.fft.fftshift(ftFields[:,0]), 'r-',\
                                                             np.fft.fftshift(angularFrequencies), np.fft.fftshift(ftFields[:,1]), 'g-',\
                                                             np.fft.fftshift(angularFrequencies), np.fft.fftshift(ftFields[:,2]), 'b')
plt.legend(("FEx", "FEy", "FEz"), loc='best')

In [ ]:
plt.close()

In [ ]:
plt.plot(np.fft.fftshift(frequencies/1e9), np.fft.fftshift(np.linalg.norm(ftFields, axis=1)**2), 'r-')

In [ ]:
grid.fldE.shape

In [ ]:
plt.close()
plt.plot(np.fft.fftshift(frequencies/1e9), np.fft.fftshift(abs(np.sum(ftFields, axis=1))), 'r-')
plt.xlim(0,11)
plt.ylim(-0.5,50)

In [ ]:
def eigenfrequency(m, n, p, Lx, Ly, Lz):
    return c0/2 * np.sqrt( (m/Lx)**2 + (n/Ly)**2 + (p/Lz)**2 )

In [ ]:
eigenfreqList = []

xModes = 4
yModes = 3
zModes = 3

# TM modes
for m in range(1, xModes):
    for n in range(1, yModes):
        for p in range(zModes):
            eigenfreqList.append( eigenfrequency(m,n,p, Lx, Ly, Lz) )
# TE modes
for p in range(1, zModes):
    for n in range(yModes):
        if n==0:
            for m in range(1, xModes):
                eigenfreqList.append( eigenfrequency(m,n,p, Lx, Ly, Lz) )
        else:
            eigenfreqList.append( eigenfrequency(0,n,p, Lx, Ly, Lz) )

In [ ]:
plt.close()
plt.plot(np.fft.fftshift(frequencies/1e9), np.fft.fftshift(abs(np.sum(ftFields, axis=1))), 'r-')

plt.vlines([item/1e9 for item in np.unique(eigenfreqList)], 0, 50, linestyles='-.')
plt.xlim(0,10)
plt.ylim(0,50)

In [ ]:
import scipy.signal as spsig

In [ ]:
subset = abs(np.sum(ftFields, axis=1))[:len(ftFields)//2]

In [ ]:
subsetFrequencies = frequencies[:len(ftFields)//2]/1e9

In [ ]:
peaks, _ = spsig.find_peaks(abs(np.sum(ftFields, axis=1))[:len(ftFields)//2])

In [ ]:
plt.close()
plt.plot(np.fft.fftshift(frequencies/1e9), np.fft.fftshift(abs(np.sum(ftFields, axis=1))), 'r-')
plt.plot(np.fft.fftshift(frequencies/1e9), abs(np.sum(ftFields, axis=1)), 'r-')

plt.plot(subsetFrequencies[peaks], subset[peaks], "x")

plt.vlines([item/1e9 for item in np.unique(eigenfreqList)], 0, 50, linestyles='-.')
plt.xlim(0,10)
plt.ylim(0,50)

In [ ]:
np.unique(subsetFrequencies[peaks])[:len(np.unique(eigenfreqList))]

In [ ]:
np.unique(eigenfreqList)/1e9

## Plot field slices using Holoviews

In [ ]:
hv.extension('matplotlib')

In [ ]:
dataArray = grid.fldE[0,:,:,:]

In [ ]:
dataArray.shape

In [ ]:
ds = hv.Dataset((np.arange(dataArray.shape[2]), np.arange(dataArray.shape[1]), np.arange(dataArray.shape[0]), dataArray ), ['x', 'y', 'z'], "z-component")

In [ ]:
hv.opts.defaults(
hv.opts.Image(cmap='viridis', fig_size=200.0, normalize=True, colorbar=True),
hv.opts.Labels(fontsize='8pt'),
    hv.opts.Path(color='white'),
    hv.opts.Spread(linewidth=600)
)

In [ ]:
ds.to(hv.Image, ['x','y'])

## Point charge field init

In [ ]:
def pointCharge(x,y,z):
    factor = 1.0/(4*np.pi*eps0)
    return factor * np.array([x,y,z])/np.linalg.norm(np.array([x,y,z]))**3

In [ ]:
grid2 = UnifiedYeeGrid(Nx, Ny, Nz, pmlCells=(0,0,0), cellDim=(1./Cx, 1./Cy, 1./Cz) )

In [ ]:
grid2.initE(pointCharge)

In [ ]:
dataArray2 = np.linalg.norm(grid2.fldE, axis=0)

In [ ]:
from matplotlib.colors import LogNorm, SymLogNorm

In [ ]:
vmin = max(1e-40, abs(dataArray2).min())
vmax = max(1e-10, abs(dataArray2).max())

linthrsh = np.quantile(abs(dataArray2), 0.25)
forceNorm = SymLogNorm(vmax=vmax, vmin=-vmax, linthresh=linthrsh );

In [ ]:
hv.opts.defaults(
hv.opts.Image(cmap='viridis', fig_size=200.0, normalize=True, colorbar=True, norm=forceNorm),
hv.opts.Labels(fontsize='8pt'),
    hv.opts.Path(color='white'),
    hv.opts.Spread(linewidth=600)
)

In [ ]:
ds2 = hv.Dataset((np.arange(grid2.originCellCoordinates[2], grid2.originCellCoordinates[2]+grid2.dz*grid2.Nz_virt, grid2.dz),
                  np.arange(grid2.originCellCoordinates[1], grid2.originCellCoordinates[1]+grid2.dy*grid2.Ny_virt, grid2.dy),
                  np.arange(grid2.originCellCoordinates[0], grid2.originCellCoordinates[0]+grid2.dx*grid2.Nx_virt, grid2.dx), abs(dataArray2) ), ['x', 'y', 'z'], "$\\vert \\vec{E}\\vert$ [V/m]", fontsize='large')

In [ ]:
ds2.to(hv.Image, ['x','y'])